given following packages
---
- `gdal` `2.4.4`
- `rasterio` `1.1.5`
- `proj 7.0.0`
- `pyproj` `2.6.1.post1`
- `h5py` `2.10.0`

In [1]:
import numpy as np
import h5py
import rasterio
from rasterio.transform import from_origin
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [2]:
# load KNMI radar data
filename_h5 = r'RAD_NL25_PCP_NA_201910281900.h5'

# Filename h5 converted to tif
filename_rad = 'RAD_NL25_PCP_NA_201910281900_rad.tif'

# Filename tif reprojected to RD
filename_rd = 'RAD_NL25_PCP_NA_201910281900_rd_rio.tif'

In [3]:
# Open h5 file into numpy array
f = h5py.File(filename_h5, 'r')
im = np.copy(f['image1']['image_data'])
im = im.astype(np.float32)
f.close()
arr = np.ma.masked_where(im==255., im)

In [4]:
# Save array to tif
transform = from_origin(0, -3650, 1, 1)
new_dataset = rasterio.open(filename_rad, 'w', driver='GTiff',
                            height = arr.shape[0], width = arr.shape[1],
                            count=1, dtype=str(arr.dtype),
                            crs='+proj=stere +x_0=0 +y_0=0 +lat_0=90 +lon_0=0 +lat_ts=60 +a=6378.137 +b=6356.752',                            
                            transform=transform)

new_dataset.write(arr, 1)
new_dataset.close()

In [5]:
# Reproject to RD projection
dst_crs = rasterio.crs.CRS.from_proj4('+proj=sterea +lat_0=52.15616055555555 +lon_0=5.38763888888889 +k=0.9999079 +x_0=155000 +y_0=463000 +ellps=bessel +units=m +no_defs +<>')
#dst_crs = rasterio.crs.CRS.from_epsg(4326)

with rasterio.open(filename_rad) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    with rasterio.open(filename_rd, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)

In [6]:
!gdalinfo RAD_NL25_PCP_NA_201910281900_rd_rio.tif

Driver: GTiff/GeoTIFF
Files: RAD_NL25_PCP_NA_201910281900_rd_rio.tif
Size is 768, 826
Coordinate System is:
PROJCS["unnamed",
    GEOGCS["Bessel 1841",
        DATUM["unknown",
            SPHEROID["bessel",6377397.155,299.1528128]],
        PRIMEM["Greenwich",0],
        UNIT["degree",0.0174532925199433]],
    PROJECTION["Oblique_Stereographic"],
    PARAMETER["latitude_of_origin",52.15616055555555],
    PARAMETER["central_meridian",5.38763888888889],
    PARAMETER["scale_factor",0.9999079],
    PARAMETER["false_easting",155000],
    PARAMETER["false_northing",463000],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]]]
Origin = (-236275.402756847790442,900792.860893698176369)
Pixel Size = (961.025429589943087,-961.025429589943087)
Metadata:
  AREA_OR_POINT=Area
Image Structure Metadata:
  INTERLEAVE=BAND
Corner Coordinates:
Upper Left  ( -236275.403,  900792.861) (  0d52'23.92"W, 55d55'59.21"N)
Lower Left  ( -236275.403,  106985.856) (  0d 3'31.53"E, 48d49'46.56"N)
Upper Right (  